# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
url = 'http://www.indeed.com/jobs?q=data+scientist&l=nyc&start=30'

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import bs4
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import statsmodels.api as sm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [4]:
#This function scrapes the location from the given url

def location(soupy):
    loc = soupy.findAll(class_='location')

    locations = []
    for i in loc:
        try:
            locations.append(i.text)
        except:
            locations.append(np.nan)
    return locations

locations = location(soup)

In [5]:
#This function scrapes the company name from the given url

def company(soupy):
    company = soupy.findAll(class_='company')
    companies = []
    for i in company:
        try:
            companies.append(i.text.strip().encode('ascii','ignore'))
        except:
            companies.append(np.nan)
    return companies

companies = company(soup)

In [6]:
#This function scrapes the location from the given url

def job(soupy):
    title = soupy.findAll(class_='jobtitle')
    titles = []
    for i in title:
        try:
            titles.append(i.text.strip().encode('ascii','ignore'))
        except:
            titles.append(np.nan)
    return titles
    
titles = job(soup)

In [7]:
#This function scrapes the salary from the given url

def salary(soupy):
    salary = soupy.findAll('nobr' or 'sjcl')
    salaries = []
    for i in salary:
        try:
            salaries.append(i.get_text().strip().encode('ascii','ignore'))
        except:
            salaries.append(np.nan)
    return salaries

salaries = salary(soup)

In [8]:
#Adds the pulled data into a dataframe

df = pd.DataFrame([titles, companies, locations, salaries]).T
df.columns = ['title', 'company', 'locations','salary']
df.head()

,title,company,locations,salary
0,Data Scientist & Analytics Developer,Lancer Insurance Company,"Long Beach, NY",relevance -\n date
1,Data Scientist,Princeton Consulting,"New York, NY","$115,000 - $150,000 a year"
2,Data Scientist,Crisis Text Line,"New York, NY",About
3,Data Scientist,Dia&Co,"New York, NY",None
4,Data Scientist,x.ai,"New York, NY 10004 (Financial District area)",None


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [9]:
#cycles through each of the cities listed and the results pages pulling the job data and appending to results

url_template = 'http://www.indeed.com/jobs?q=data+scientist&l={}&start={}'

max_results_per_city = 100

results = [[],[],[],[]]

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Miami', 'Atlanta','Los+Angeles', 'Chicago'\
                 'Houston','Philadelphia','Phoenix','Houston','San+Diego']):
    for start in range(0, max_results_per_city, 10):
        response = requests.get(url_template.format(city,start))
        page = response.text
        soup = BeautifulSoup(page,'lxml')
        results[0].extend(location(soup))
        results[1].extend(company(soup))
        results[2].extend(job(soup))
        results[3].extend(salary(soup))
        pass

#reduce(lambda x, y: x.extend(y), results)


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [10]:
#creates dataframe df with all of the results for each city

df = pd.DataFrame([results[2], results[1], results[0], results[3]]).T
df.columns = ['title', 'company', 'location','salary']
df.head()

,title,company,location,salary
0,Computational Scientist,MD ANDERSON CANCER CENTER,"Houston, TX",relevance -\n date
1,Data Scientist Predictive Analytics,Gallup,"Houston, TX",About
2,Machine Learning Expert,Arundo Analytics,"Houston, TX",relevance -\n date
3,Data Scientist,Panton,"Houston, TX","$160,000 a year"
4,Senior Data Scientist,Shell,"Houston, TX",About


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [11]:
#cleans salary data by turning alpha strings to nans and dropping any salaries not in annual terms

def salarydrop(value):
    c=0
    #value = value.encode('utf8')
    value = value.split()
    if 'relevance' in value: 
        return 'nan'
    for i in value:
        if i.isalpha() == True:
            c+=1
            if c == len(value):
                return 'nan'
            
    todrop = ['hourly','hour', 'weekly','week','monthly','month','day','daily']
    for i in todrop:
        if i in value:
            return 'drop'
    return " ".join(value)
        
    
df['salary'] = df['salary'].astype(str)
df['salary'] = df['salary'].to_frame().applymap(salarydrop)
df = df[df['salary']!='drop']
df.salary.unique()

array(['nan', '$160,000 a year', '$49,506 a year', '$70,000 a year',
       '$93,431 a year', '$51,600 - $77,400 a year',
       '$60,000 - $75,000 a year', '$54,590 a year',
       '$85,000 - $115,000 a year', '$90,000 - $100,000 a year',
       '$48,000 a year', '$63,696 - $94,557 a year',
       '$50,000 - $55,000 a year', '$71,000 a year',
       '$115,000 - $150,000 a year', '$80,000 - $110,000 a year',
       '$125,000 a year', '$85,000 a year', '$110,000 a year',
       '$120,000 a year', '$180,000 a year', '$80,000 - $120,000 a year',
       '$150,000 - $180,000 a year', '$130,000 a year',
       '$170,000 - $200,000 a year', '$100,000 - $200,000 a year',
       '$45,000 a year', '$200,000 - $300,000 a year',
       '$80,000 - $200,000 a year', '$40,000 a year',
       '$100,000 - $120,000 a year', '$150,000 a year',
       '$135,000 - $165,000 a year', '$90,000 - $130,000 a year',
       '$53,900 a year', '$32,900 a year', '$55,000 - $65,000 a year',
       '$38,750 - $62,000 

In [12]:
df.head()

,title,company,location,salary
0,Computational Scientist,MD ANDERSON CANCER CENTER,"Houston, TX",nan
1,Data Scientist Predictive Analytics,Gallup,"Houston, TX",nan
2,Machine Learning Expert,Arundo Analytics,"Houston, TX",nan
3,Data Scientist,Panton,"Houston, TX","$160,000 a year"
4,Senior Data Scientist,Shell,"Houston, TX",nan


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [13]:
#removes the '$' from salary and returns a float. If salary is listed as range, the mean will be returned

def salaryfix(value):
    c=0
    value = str(value)
    if value.isalpha == True:
        return value
    if "a year" in value:
        value = value[:-6]
    if "-" in value:
        value = value.split('-')
        for nums in value:
            value = salaryfix(nums)
            c+=1
            if c==1:
                thesum=0
            thesum += value
        return thesum/2
    value = str(value)
    value = value.replace('$', '')
    value = value.replace(',', '')
    value = float(value)
    return value

df['salary'] = df['salary'].to_frame().applymap(salaryfix)

In [14]:
#this function removes any characters after the ',' in the city column

def cityfix(value):
    value = str(value)
    value = value.split(',')
    return value[0]

df['location'] = df['location'].to_frame().applymap(cityfix)

In [15]:
df = df.drop_duplicates()
df.head()

,title,company,location,salary
0,Computational Scientist,MD ANDERSON CANCER CENTER,Houston,NaN
1,Data Scientist Predictive Analytics,Gallup,Houston,NaN
2,Machine Learning Expert,Arundo Analytics,Houston,NaN
3,Data Scientist,Panton,Houston,160000.0
4,Senior Data Scientist,Shell,Houston,NaN


### Save your results as a CSV

In [16]:
#df.to_csv('jobresults.csv')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [17]:
df2 = pd.read_csv('jobresults.csv')

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [18]:
#this function returns a true value if the salary is higher than the median


median = df.salary.median()

def salarybool(value):
    if value >= median:
        return True
    else: return False
    
df['salaryhigh'] = df.salary.to_frame().applymap(salarybool)
df.head(10)

,title,company,location,salary,salaryhigh
0,Computational Scientist,MD ANDERSON CANCER CENTER,Houston,NaN,False
1,Data Scientist Predictive Analytics,Gallup,Houston,NaN,False
2,Machine Learning Expert,Arundo Analytics,Houston,NaN,False
3,Data Scientist,Panton,Houston,160000.0,True
4,Senior Data Scientist,Shell,Houston,NaN,False
5,Research Scientist,"GeoControl Systems, Inc.",Houston,NaN,False
6,Marketing Data Scientist,Alert Logic,Houston,49506.0,False
7,Data Scientist - Houston,"Amazon Web Services, Inc.",Houston,NaN,False
8,Data Scientist,IHS,Houston,NaN,False
9,Research Statistical Analyst,MD ANDERSON CANCER CENTER,Houston,70000.0,False


#### Thought experiment: What is the baseline accuracy for this model?

The baselin

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [19]:
#location, salary, and the boolean salary variable have been pulled out and dummy variables created for each location.


tempdf = df[df['salary']>0]
locationdf = tempdf[['location','salary','salaryhigh']]


locationdf['key'] = range(len(locationdf['location']))
locationdf = pd.concat([locationdf.drop('key',axis=1),pd.get_dummies(locationdf['location'])], axis = 1)
locationdf['constant'] = 1
del locationdf['Phoenix']
locationdf

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,location,salary,salaryhigh,Chicago,Glendale,Houston,Scottsdale,Spring,Sugar Land,Tempe,constant
3,Houston,160000.0,True,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
6,Houston,49506.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
9,Houston,70000.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
10,Houston,93431.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
19,Houston,64500.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
20,Houston,67500.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
23,Houston,54590.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
32,Houston,100000.0,True,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
35,Houston,95000.0,True,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
36,Houston,48000.0,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1


In [20]:
#sets target and data variables and prints shape

X = locationdf[list(locationdf.columns[3:])]
y = locationdf.salaryhigh
print X.shape
print y.shape

(62, 8)
(62,)


In [21]:
#fits statsmodel logistic regression model based on test and target data

model = sm.Logit(y,X)
model_fit = model.fit(method = 'bfgs')
model_fit.summary()

         Current function value: 0.564322
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             salaryhigh   No. Observations:                   62
Model:                          Logit   Df Residuals:                       54
Method:                           MLE   Df Model:                            7
Date:                Thu, 07 Jul 2016   Pseudo R-squ.:                  0.1852
Time:                        08:36:26   Log-Likelihood:                -34.988
converged:                      False   LL-Null:                       -42.943
                                        LLR p-value:                   0.02595
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Chicago      -14.1516    906.343     -0.016      0.988     -1790.550  1762.247
Glendale      13.6070    588.027      0.023      0.982     -1138.904  1166.118
Houston        0.5440      0.663      0.821      0.412        -0.755     1.843
Scottsdale    -1.7823      1.204     -1.480      0.139        -4.142     0.578
Spring         9.5473    109.244      0.087      0.930      -204.568   223.662
Sugar Land    13.6070    588.027      0.023      0.982     -1138.904  1166.118
Tempe          0.1575      1.520      0.104      0.917        -2.821     3.136
constant      -0.1604      0.556     -0.288      0.773        -1.251     0.930
==============================================================================
"""

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [22]:
#this function adds a field for title level and returns level based on key words in title

def titlelevel(value):
    value = str(value)
    value = value.encode('ascii','ignore')
    value = str(value)
    if 'Manager' in value:
        return 'manager'
    elif 'Director' in value:
        return 'director'
    elif 'VP' in value:
        return 'vp'
    elif 'Chief' in value:
        return 'chief'
    elif 'Analyst' in value:
        return 'analyst'
    elif 'Scientist' in value:
        return 'scientist'
    elif 'Engineer' in value:
        return 'engineer'
    else: return 'other'

df['titlelevel'] = df.title.to_frame().applymap(titlelevel)

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [23]:
# import patsy
# f = 'y ~ x'
# y, X = patsy.dmatrices(f, df, return_type='dataframe')
# sm.Logit(y, X).fit().summary()

In [39]:
#titlelevel, salary, and the boolean salary variable have been pulled out and dummy variables created for each title level.


tempdf = df[df['salary']>0]
titledf = tempdf[['titlelevel','salary','salaryhigh']]
titledf['key'] = range(len(titledf['titlelevel']))
titledf = pd.concat([titledf.drop('key',axis=1),pd.get_dummies(titledf['titlelevel'])], axis = 1)
titledf['constant'] = 1
del titledf['analyst']
titledf.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,titlelevel,salary,salaryhigh,engineer,manager,other,scientist,constant
3,scientist,160000.0,True,0.0,0.0,0.0,1.0,1
6,scientist,49506.0,False,0.0,0.0,0.0,1.0,1
9,analyst,70000.0,False,0.0,0.0,0.0,0.0,1
10,scientist,93431.0,False,0.0,0.0,0.0,1.0,1
19,manager,64500.0,False,0.0,1.0,0.0,0.0,1


In [40]:
#fits scikit-learn logistic regression model based on train and test data as well as prints the confusion matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)
lr = LogisticRegression(solver='liblinear')
lr_model = lr.fit(X_train, y_train)
lr_ypred = lr_model.predict(X_test)
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,False,True
False,11,0
True,0,10


In [41]:
print(classification_report(y_test, lr_ypred, labels=lr.classes_))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00        11
       True       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00        21



In [42]:
cross_val_score(lr, X, y, cv=3).mean()

1.0

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [43]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [44]:
pdf = pd.DataFrame(X, columns=X.columns)
pdf['Target'] = y
pdf.head()

,Chicago,Glendale,Houston,Scottsdale,Spring,Sugar Land,Tempe,constant,Target
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,True
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,False
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,False
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,False
19,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,False


In [45]:
lr1 = LogisticRegression(penalty='l1')
lr2 = LogisticRegression(penalty='l2')

lr1_model = lr1.fit(X_train, y_train)
lr2_model = lr2.fit(X_train, y_train)
y1_pred = lr1.predict(X_test)
y2_pred = lr2.predict(X_test)

In [46]:
cm1 = confusion_matrix(y_test, y1_pred, labels=lr1.classes_)
cm1 = pd.DataFrame(cm1, columns=lr1.classes_, index=lr1.classes_)

cm2 = confusion_matrix(y_test, y2_pred, labels=lr2.classes_)
cm2 = pd.DataFrame(cm2, columns=lr2.classes_, index=lr2.classes_)

In [47]:
cm1

,False,True
False,11,0
True,0,10


In [48]:
cm2

,False,True
False,11,0
True,0,10


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

In [49]:
tempdf.company.unique

<bound method Series.unique of 3                                         Panton
6                                    Alert Logic
9                      MD ANDERSON CANCER CENTER
10                                  Schlumberger
19                              Oscar Technology
20                             Houston Methodist
23                                        Gallup
32                    Baylor College of Medicine
35                       Earthstream Global, Inc
36                             Houston Methodist
39                                Selby Jennings
42                           The Hanover Company
57                                           HX5
58                                        Gallup
59                     MD ANDERSON CANCER CENTER
72                                        Gallup
75                                        Gallup
78                   McKinley Marketing Partners
79                      PharMEDium Services, LLC
86                             Houston

In [50]:
#this function adds a field for title level and returns level based on key words in title

def comkeyword(value):
    value = value.encode('ascii','ignore')
    value = str(value)
    value = value.encode('ascii','ignore')
    if 'Corp' in value:
        return 'corp'
    elif 'Analytics' in value:
        return 'analytics'
    elif 'University' in value:
        return 'university'
    elif 'Group' in value or 'group' in value:
        return 'group'
    elif 'Technology' in value or 'Technologies' in value:
        return 'tech'
    elif 'Software' in value:
        return 'software'
    elif 'Engineer' in value:
        return 'engineer'
    else: return 'other'

df['keyword'] = df.company.to_frame().applymap(comkeyword)

In [51]:
df

,title,company,location,salary,salaryhigh,titlelevel,keyword
0,Computational Scientist,MD ANDERSON CANCER CENTER,Houston,NaN,False,scientist,other
1,Data Scientist Predictive Analytics,Gallup,Houston,NaN,False,scientist,other
2,Machine Learning Expert,Arundo Analytics,Houston,NaN,False,other,analytics
3,Data Scientist,Panton,Houston,160000.0,True,scientist,other
4,Senior Data Scientist,Shell,Houston,NaN,False,scientist,other
5,Research Scientist,"GeoControl Systems, Inc.",Houston,NaN,False,scientist,other
6,Marketing Data Scientist,Alert Logic,Houston,49506.0,False,scientist,other
7,Data Scientist - Houston,"Amazon Web Services, Inc.",Houston,NaN,False,scientist,other
8,Data Scientist,IHS,Houston,NaN,False,scientist,other
9,Research Statistical Analyst,MD ANDERSON CANCER CENTER,Houston,70000.0,False,analyst,other


In [54]:
tempdf = df[df['salary']>0]
titledf = tempdf[['titlelevel','salary','salaryhigh','keyword']]
titledf['key'] = range(len(titledf['titlelevel']))
titledf = pd.concat([titledf.drop('key',axis=1),pd.get_dummies(titledf['titlelevel'])], axis = 1)
titledf['key'] = range(len(titledf['keyword']))
titledf = pd.concat([titledf.drop('key',axis=1),pd.get_dummies(titledf['keyword'])], axis = 1)
titledf['constant'] = 1
del titledf['analyst']
titledf.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,titlelevel,salary,salaryhigh,keyword,engineer,manager,other,scientist,corp,group,other,software,tech,university,constant
3,scientist,160000.0,True,other,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
6,scientist,49506.0,False,other,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
9,analyst,70000.0,False,other,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
10,scientist,93431.0,False,other,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
19,manager,64500.0,False,tech,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)
lr = LogisticRegression(solver='liblinear')
lr_model = lr.fit(X_train, y_train)
lr_ypred = lr_model.predict(X_test)
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,False,True
False,11,0
True,0,10


In [56]:
print(classification_report(y_test, lr_ypred, labels=lr.classes_))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00        11
       True       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00        21



In [57]:
cross_val_score(lr, X, y, cv=3).mean()

1.0

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

In [65]:
newdf = titledf.head(100)
thesalaries = newdf['salary']
newdf.head()
newdf.shape

(62, 15)

In [67]:
print X.shape
print y.shape
lr_ypred = lr_model.predict(X)
#lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
#lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
#lr_cm
lr_ypred

(62, 9)
(62,)


array([ True, False, False, False, False, False, False,  True,  True,
       False, False, False, False,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
       False, False, False, False,  True,  True, False, False, False,
       False, False,  True, False, False, False,  True,  True, False,
       False,  True, False,  True,  True,  True, False, False], dtype=bool)